## Descripción de los datos

***books:***

Contiene datos sobre libros:

- book_id — identificación del libro
- author_id — identificación del autor o autora
- title — título
- num_pages — número de páginas
- publication_date — fecha de la publicación
- publisher_id — identificación de la editorial

***authors:***

Contiene datos sobre autores:

- author_id — identificación del autor o autora
- author — el autor o la autora

***publishers:***

Contiene datos sobre editoriales:

- publisher_id — identificación de la editorial
- publisher — la editorial

***ratings:***

Contiene datos sobre las calificaciones de usuarios:

- rating_id — identificación de la calificación
- book_id — identificación del libro
- username — el nombre del usuario que revisó el libro
- rating — calificación

***reviews:***

Contiene datos sobre las reseñas de los y las clientes:

- review_id — identificación de la reseña
- book_id — identificación del libro
- username — el nombre del usuario que revisó el libro
- text — el texto de la reseña

# Objetivos del estudio

Durante la pandemia por COVID-19, el mundo entero experimentó un cambio radical en sus hábitos de consumo de tiempo libre. Las salidas al cine, cafés y centros comerciales fueron reemplazadas por actividades dentro del hogar, y entre ellas, la lectura brilló con fuerza. Este contexto abrió la puerta para que startups tecnológicas buscaran posicionarse con aplicaciones dirigidas a los amantes de los libros.

El objetivo de este proyecto es analizar los datos proporcionados por un servicio en este mercado emergente. La base de datos incluye información sobre libros, autores, editoriales, calificaciones y reseñas de usuarios. A través del análisis de estos datos, se busca:

- Identificar patrones clave en la publicación y recepción de libros.
- Descubrir a los autores, editoriales y usuarios más influyentes.
- Establecer bases sólidas para diseñar una propuesta de valor atractiva para una nueva aplicación dedicada a lectores.

En otras palabras, vamos a escarbar entre letras, números y estrellas (de calificación, claro) para entender qué mueve al lector moderno, ver cómo podese podría conquistar este mercado y generar una propuesta de valor para un nuevo producto.

# Configuración inicial

In [66]:
#Biblioteca a importar
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [67]:
#Conexión a la base de datos
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')


# Análisis inicial

In [68]:
#Mostramos las primeras 5 filas de cada tabla
#Creamos una lista con los nombres de las tablas que queremos consultar
tables = ['books', 'authors', 'publishers', 'ratings', 'reviews']

#Iteramos sobre la lista de tablas y mostramos las primeras 5 filas de cada una
for table in tables:
    print(f"Tabla: {table}")
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5", con=engine)
    display(df)

Tabla: books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Tabla: authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Tabla: publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Tabla: ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Tabla: reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


    - Después de dar un vistazo a las primeras filas de los datos encontramos a la tabla books como la tabla central ya que contiene la mayor cantidad de llaves foraneas, mientras que el resto son mayormente de consulta.
    - Me llama la atención particularmente que ratings y reviews no estén conectados por una llave foranea enfocada los usuarios que dejan una calificación y reseña, a la larga esto entorpecera futuras consultas relacionadas con los usuarios que participan activamente.

# Consultas Específifcas

## Número de libros publicados después del 1 de enero de 2000

In [56]:
#Generamos un string con la consulta SQL para contar los libros publicados después del 1 de enero de 2000 (year-month-day)
query_1 = """SELECT COUNT(*) AS publicados_desp_ene2000 FROM books WHERE publication_date > '2000-01-01';"""

display("Numero de libros publiados después del 1 de enero de 2020:", pd.read_sql(query_1, con=engine))

'Numero de libros publiados después del 1 de enero de 2020:'

,publicados_desp_ene2000
0,819


    - De los 1000 libros registrados en la BD encontramos que más del 80% fueron publicados después del 2000, por lo que confirmamos que en este estudio de mercado la preferencia recae en la literatura moderna.

## Número de reseñas de usuarios y la calificación promedio para cada libro

In [55]:
#Generamos un string con la consulta SQL para contar el número de reseñas de usuarios y la calificación promedio para cada libro
#Usamos LEFT JOIN para asegurarnos de incluir libros sin reseñas o calificaciones
#Agrupamos por book_id y title para obtener el conteo de reseñas y la calificación promedio
#Ordenamos por calificación promedio de forma descendente para ver los libros mejor valorados
query_2 = """
SELECT 
    b.book_id,
    b.title,
    COUNT(DISTINCT rv.review_id) AS contador_resenas,
    ROUND(AVG(rt.rating), 2) AS calificacion_promedio
FROM books b
LEFT JOIN reviews rv ON b.book_id = rv.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY calificacion_promedio DESC;
"""

#Creamos una nueva consulta para calcular el promedio de reseñas  y calificación para todos los libros
query_2_2 = """
SELECT 
    ROUND(AVG(contador_resenas),1) AS total_contador_resenas,
    ROUND(AVG(calificacion_promedio),1) AS total_calificacion_promedio
FROM (
    SELECT 
        b.book_id,
        COUNT(DISTINCT rv.review_id) AS contador_resenas,
        ROUND(AVG(rt.rating), 2) AS calificacion_promedio
    FROM books b
    LEFT JOIN reviews rv ON b.book_id = rv.book_id
    LEFT JOIN ratings rt ON b.book_id = rt.book_id
    GROUP BY b.book_id, b.title
) AS subquery;
"""

display("Tabla con ID, título, conteo de reseñas y calificación promedio:", pd.read_sql(query_2, con=engine))
display("Promedio de número de reseñas y calificación promedio de todos los libros:", pd.read_sql(query_2_2, con=engine))

'Tabla con ID, título, conteo de reseñas y calificación promedio:'

,book_id,title,contador_resenas,calificacion_promedio
0,86,Arrows of the Queen (Heralds of Valdemar #1),2,5.00
1,901,The Walking Dead Book One (The Walking Dead #...,2,5.00
2,390,Light in August,2,5.00
3,972,Wherever You Go There You Are: Mindfulness Me...,2,5.00
4,136,Captivating: Unveiling the Mystery of a Woman'...,2,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,3,2.25
996,316,His Excellency: George Washington,2,2.00
997,202,Drowning Ruth,3,2.00
998,371,Junky,2,2.00


'Promedio de número de reseñas y calificación promedio de todos los libros:'

,total_contador_resenas,total_calificacion_promedio
0,2.8,3.9


    - Al generar la consulta encontramos que practicamente todos los libros con calificación promedio perfecta de 5 cuentan con solo 2 reseñas, esta tendencia indica que los resultados podrían ser poco confiables y sería necesario contar con mayor cantidad de usuarios dejando sus reseñas y calificaciones para confirmar si en serio existe un alto nivel de agrado entre los lectores.
    - Como consulta adicional calculamos el promedio de reseñas y el promedio global de calificación de los libros; siendo que la cantidad de reseñas que los usuarios exponen por libro es de 2.8 se podría colocar un filtro de que la calificación de un libro sea valida a partír de recibir al menos 3 reseñas.
    - De igual forma al no conocer la confiabilidad en las calificaciones no podemos confirmar si este promedio de 3.9 de todos los libros es certero.

## Editorial con el mayor número de libros publicados (con más de 50 páginas)

In [52]:
#Generamos un string con la consulta SQL para encontrar la editorial con el mayor número de libros publicados (con más de 50 páginas)
query_3 = """
SELECT 
    p.publisher,
    COUNT(b.book_id) AS contador_libros
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY contador_libros DESC
LIMIT 5;
"""

display("Editorial con más publicaciones (+50 páginas):", pd.read_sql(query_3, con=engine))

'Editorial con más publicaciones (+50 páginas):'

,publisher,contador_libros
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


    - Identificamos a la editorial PENGUIN BOOKS como la que ha publicado la mayor cantidad de libros con más de 50 páginas, estando muy por encima del segundo lugar VINTAGE que cuenta con solo 31 libros y del tercero GRAND CENTRAL PUBLISHING con solo 25.
    - Es curioso que si revisamos el cuarto lugar sea PENGUIN CLASSICS que por el nombre suponemos que es una divisiń de PENGUIN BOOKS pero que se dedica a publicar literatura clásica.

## Autor con la más alta calificación promedio del libro (al menos 50 calificaciones)

In [50]:
#Generamos un string con la consulta SQL para encontrar el autor con la calificación promedio más alta, considerando solo autores con al menos 50 calificaciones
#Usamos JOIN para combinar las tablas ratings, books y authors
#Agrupamos por autor y calculamos el promedio de calificaciones y el conteo de calificaciones
#Filtramos para incluir solo autores con al menos 50 calificaciones
#Ordenamos por calificación promedio de forma descendente y limitamos a 1 para obtener el autor con la calificación más alta
query_4 = """
SELECT 
    a.author,
    ROUND(AVG(rt.rating), 2) AS promedio_autor_calificacion,
    COUNT(rt.rating_id) AS contador_calificaciones
FROM ratings rt
JOIN books b ON rt.book_id = b.book_id
JOIN authors a ON b.author_id = a.author_id
GROUP BY a.author
HAVING COUNT(rt.rating_id) >= 50
ORDER BY promedio_autor_calificacion DESC
LIMIT 5;
"""

display("Autor con la califcación promedio más alta (al menos 50 calificaciones recibidas):", pd.read_sql(query_4, con=engine))

'Autor con la califcación promedio más alta (al menos 50 calificaciones recibidas):'

,author,promedio_autor_calificacion,contador_calificaciones
0,Diana Gabaldon,4.30,50
1,J.K. Rowling/Mary GrandPré,4.29,312
2,Agatha Christie,4.28,53
3,Markus Zusak/Cao Xuân Việt Khương,4.26,53
4,J.R.R. Tolkien,4.24,166


    - Los primeros 3 lugares se encuentran separados en su calificación promedio por solo 0.1, siendo DIANA GABALDON la autorá con la mejor calificación, seguida de J.K. Rowling y en tercero Agatha Christie.
    - Cabe mencionar que solo las primeras 2 autoras caen en la clasificación de literatura moderna, mientras que Agatha Cristie es literatura clásica.
    - Dentro del top 5 de autores solo 2 cuentan con más de 100 calificaciones registradas por los usuarios, ambos grandes escritores del género de fantasía: J.K. Rowling y J.R.R. Tolkien. De ahí, para el promedio de calificaciones para el resto de autores ronda las 50.

## Promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros

In [57]:
#Generamos un string con la consulta SQL para encontrar la promedio de reseñas de texto entre los usuarios que han calificado más de 50 libros
#Usamos una CTE (Common Table Expression) para identificar a los usuarios que han calificado más de 50 libros
# Luego, unimos esta CTE con la tabla de reseñas para contar las reseñas de texto de esos usuarios
#Finalmente, calculamos el promedio de reseñas de texto y redondeamos el resultado a dos decimales
query_5 = """
WITH usuarios_calificadores AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(DISTINCT book_id) > 50
)
SELECT 
    ROUND(AVG(contador_resenas), 2) AS promedio_resenas
FROM  (
    SELECT u.username, COUNT(r.review_id) AS contador_resenas
    FROM usuarios_calificadores u
    LEFT JOIN reviews r ON u.username = r.username
    GROUP BY u.username) AS subquery;
"""

display("Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:", pd.read_sql(query_5, con=engine))

'Promedio de reseñas de texto entre usuarios que calificaron más de 50 libros:'

,promedio_resenas
0,24.33


    - Aqui encontramos que los usuarios con más de 50 libros calificados son sumamente participativos ya que es practicamente la mitad de las veces que plasman una calificación dejan una reseña.
    - Este número está muy por encima del promedio de participación de reseñas del total de usuarios que si recordamos era del 2.8.

# Conclusiones

El análisis de la base de datos permitió descubrir tendencias clave sobre cómo se publica, consume y valora el contenido en la plataforma. Una gran parte de las obras identificadas pertenece a las últimas dos décadas, lo que refleja un catálogo dinámico y alineado con los gustos contemporáneos. Además, algunos títulos lograron destacar no solo por su popularidad en interacciones, sino también por mantener calificaciones promedio altas, lo que los convierte en referencias naturales para la curaduría de futuras recomendaciones.

En este panorama, emergen actores que concentran gran parte del peso cultural de la plataforma. Determinadas editoriales demuestran una capacidad notable para publicar obras de mayor extensión y contenido sustancial, mientras que ciertos autores mantienen de forma consistente una valoración sobresaliente respaldada por un volumen considerable de calificaciones. Del mismo modo, un grupo reducido de lectores destaca por su nivel de participación, contribuyendo de manera significativa tanto en cantidad como en calidad de reseñas.

Estos hallazgos ofrecen una base sólida para diseñar propuestas que combinen contenido relevante, relaciones estratégicas con creadores y editoriales, y una comunidad activa que impulse la interacción. Apostar por obras y autores con alta recepción, reforzar la presencia de editoriales con catálogo robusto y reconocer el papel de los usuarios más comprometidos podría sentar las bases de una aplicación que no solo distribuya libros, sino que también fomente el descubrimiento y la conversación en torno a ellos.